# 🌍 Global Market Attractiveness Analysis

This notebook analyzes key socio-economic indicators to evaluate the attractiveness of international markets using data from the World Bank and Our World in Data.

In [ ]:
# ✅ Automatically download CSV files from GitHub if running in Colab
!mkdir -p data
base_url = "https://raw.githubusercontent.com/Joel1993055/Global-Market-Attractiveness-Analysis-using-Open-Data/main/data/"

files = [
    "API_NY.GDP.MKTP.CD_DS2_en_csv_v2_370168.csv",
    "API_NY.GDP.MKTP.KD.ZG_DS2_en_csv_v2_369935.csv",
    "API_SP.POP.TOTL_DS2_en_csv_v2_370004.csv",
    "share-of-individuals-using-the-internet.csv"
]

for file in files:
    !wget -q "{base_url}{file}" -O "data/{file}"


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px

In [ ]:
df_gdp = pd.read_csv('data/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_370168.csv', skiprows=4)
df_growth = pd.read_csv('data/API_NY.GDP.MKTP.KD.ZG_DS2_en_csv_v2_369935.csv', skiprows=4)
df_population = pd.read_csv('data/API_SP.POP.TOTL_DS2_en_csv_v2_370004.csv', skiprows=4)
df_internet = pd.read_csv('data/share-of-individuals-using-the-internet.csv')

In [ ]:
latest_year = '2022'

df_gdp_clean = df_gdp[['Country Name', latest_year]].rename(columns={latest_year: 'GDP'})
df_growth_clean = df_growth[['Country Name', latest_year]].rename(columns={latest_year: 'GDP_growth'})
df_population_clean = df_population[['Country Name', latest_year]].rename(columns={latest_year: 'Population'})

df_internet_clean = df_internet[df_internet['Year'] == 2022][['Entity', 'Individuals using the Internet (% of population)']]
df_internet_clean = df_internet_clean.rename(columns={'Entity': 'Country Name', 'Individuals using the Internet (% of population)': 'Internet_Usage'})


In [ ]:
df_merged = df_gdp_clean.merge(df_growth_clean, on='Country Name')\
                         .merge(df_population_clean, on='Country Name')\
                         .merge(df_internet_clean, on='Country Name')

df_merged = df_merged.dropna()
df_merged.head()

In [ ]:
scaler = MinMaxScaler()
df_scaled = df_merged.copy()
df_scaled[['GDP_norm', 'GDP_growth_norm', 'Population_norm', 'Internet_Usage_norm']] = scaler.fit_transform(
    df_scaled[['GDP', 'GDP_growth', 'Population', 'Internet_Usage']]
)
df_scaled.head()

In [ ]:
fig = px.scatter(df_scaled, 
                 x='GDP_norm', y='Internet_Usage_norm', 
                 size='Population_norm', 
                 color='GDP_growth_norm',
                 hover_name='Country Name',
                 title='Global Market Attractiveness: GDP vs Internet Usage')
fig.show()